## Web scraping for globaltenders.com 
Domain: Asia Pacific Law And Legal Tenders

In [5]:
import os
import requests
import json
from bs4 import BeautifulSoup


In [6]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}

# Asia pacific
# Other domains can be added (eg. Information Technology Tenders, etc). The below shows legal tenders as an example.
LIST_URL = "https://www.globaltenders.com/asia-pacific/ap-legal-tenders" 

In [7]:
def get_page(url):
    """Fetch a webpage with polite sleeping."""
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return BeautifulSoup(response.text, "html.parser")

def scrape_tenders(soup):
    tenders = []
    for tender in soup.select('div.tender-wrap'):
        title = tender.select_one('.title-wrap [itemprop="name"]')
        title_text = title.get_text(strip=True) if title else ""
        
        country = tender.select_one('[itemprop="address"]')
        country_text = country.get_text(strip=True) if country else ""
        
        posting_date = tender.select_one('[itemprop="startDate"]')
        posting_date_text = posting_date.get_text(strip=True) if posting_date else ""
        
        deadline = tender.select_one('[itemprop="endDate"]')
        deadline_text = deadline.get_text(strip=True) if deadline else ""
        
        detail_link = tender.select_one('a.btn-new')
        detail_url = detail_link['href'] if detail_link else ""
        
        tenders.append({
            "title": title_text,
            "country": country_text,
            "posting_date": posting_date_text,
            "deadline": deadline_text,
            "detail_url": detail_url
        })
    for idx, tender in enumerate(tenders, start=1):
        tender["no"] = idx
    return tenders

In [8]:
if __name__ == "__main__":
    os.makedirs("output", exist_ok=True)
    soup = get_page(LIST_URL)
    # HTML page
    with open("output/listing_page.html", "w", encoding="utf-8") as f:
        f.write(str(soup))
    
    # Scraped tenders for the first page
    tenders = scrape_tenders(soup)
    with open("output/scrap_output_apac.json", "w", encoding="utf-8") as f:
        json.dump(tenders, f, ensure_ascii=False, indent=2)

Future dev: scrap for all pages